# Loading data into Syft Domain Server

### Import packages

In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft import autocache
import pandas as pd

### Launch a Syft Domain Server

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)

In [ ]:
# log into the node with default root credentials
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# List the API
domain_client.api

### Data Subjects

For this notebook, we'll create a sample dataset that includes trade information of different commodities for different countries.

In [ ]:
# Check for existing Data Subjects
data_subjects = domain_client.data_subject_registry.get_all()

In [ ]:
data_subjects

In [ ]:
assert len(data_subjects) == 0

### Add different countries as Data subjects

In [ ]:
country = sy.DataSubject(name="Country", aliases=["country_code"])

In [ ]:
canada = sy.DataSubject(name="Canada", aliases=["country_code:ca"])
germany = sy.DataSubject(name="Germany", aliases=["country_code:de"])
spain = sy.DataSubject(name="Spain", aliases=["country_code:es"])
france = sy.DataSubject(name="France", aliases=["country_code:fr"])
japan = sy.DataSubject(name="Japan", aliases=["country_code:jp"])

In [ ]:
country.add_member(canada)
country.add_member(germany)
country.add_member(spain)
country.add_member(france)
country.add_member(japan)

country.members

In [ ]:
# Adds the data subject and all its members to the registry
response = domain_client.data_subject_registry.add_data_subject(country)
response

In [ ]:
assert response

In [ ]:
# Lets look at the data subjects added to the data
data_subjects = domain_client.data_subject_registry
data_subjects

In [ ]:
assert len(data_subjects) == 10

## Adding a Dataset

For simplicity, we'll be adding only Canada's trade dataset

In [ ]:
canada_dataset_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"

In [ ]:
df = pd.read_csv(autocache(canada_dataset_url))
df

In Syft, every dataset has two variants - a Mock and a Private dataset.

* **Mock** dataset is a mock version of the private data that can be accessed & read by the data scientists.
* **Private** dataset is the actual data that will never be accessed by the data scientist.

To keep things simple, we sample different data points as Mock & Private. But in reality you would want to generate a random dataset for the Mock variant. 

In [ ]:
# private data samples
ca_data = df[0:10]
ca_data

In [ ]:
# Mock data samples
mock_ca_data = df[10:20]
mock_ca_data

### Create a Dataset

In Syft, `Dataset` is a collection of Assets. For example, `Dataset` can be a "Lung Cancer Dataset", and `Assets` will be train, test & validation splits for this dataset.

In [ ]:
dataset = sy.Dataset(name="Canada Trade Value")

In [ ]:
dataset.set_description("Canada Trade Data")

In [ ]:
dataset.add_citation("Person, place or thing")
dataset.add_url("https://github.com/OpenMined/datasets/tree/main/trade_flow")

In [ ]:
dataset.add_contributor(name="Andrew Trask", 
                        email="andrew@openmined.org",
                        note="Andrew runs this domain and prepared the dataset metadata.")

dataset.add_contributor(name="Madhava Jay", 
                        email="madhava@openmined.org",
                        note="Madhava tweaked the description to add the URL because Andrew forgot.")

In [ ]:
dataset.contributors

In [ ]:
assert len(dataset.contributors) == 2

### Add Assets to the Dataset

In [ ]:
ctf = sy.Asset(name="canada_trade_flow")
ctf.set_description("Canada trade flow represents export & import of different commodities to other countries")

In [ ]:
ctf.add_contributor(name="Andrew Trask", 
                    email="andrew@openmined.org",
                    note="Andrew runs this domain and prepared the asset.")

In [ ]:
# This is where we add the private data (pandas df/numpy array) to the `Asset`
ctf.set_obj(ca_data)

In [ ]:
# We must set the shape of this private data
ctf.set_shape(ca_data.shape)

In [ ]:
# We assign the data subject for whom this data belongs to, in this
ctf.add_data_subject(canada)

In [ ]:
# Optionally, if we don't want to add any Mock dataset
ctf.no_mock()

In [ ]:
# We must add this Asset to our Dataset
dataset.add_asset(ctf)

In [ ]:
# In case we want to remove a dataset & its associated assets
dataset.remove_asset(name=ctf.name)

In [ ]:
# Let's assign the Mock data to the Asset by calling `set_mock` method
ctf.set_mock(mock_ca_data)

In [ ]:
# Let's add our Asset back into our "Canada Trade Value" Dataset
dataset.add_asset(ctf)

### Upload Dataset to Domain Server

In [ ]:
domain_client.upload_dataset(dataset)

In [ ]:
# We can list all the datasets on the Domain Server by invoking the following
datasets = domain_client.datasets
datasets

In [ ]:
assert len(datasets) == 1

In [ ]:
datasets

### Reading the Dataset from Domain Server

Following the logical hierarchy of `Dataset`, `Asset`, and its variant, we can read the data as follows

In [ ]:
# Reading the mock dataset
mock = domain_client.datasets[0].assets[0].mock

In [ ]:
assert mock_ca_data.equals(mock)

In [ ]:
# Reading the real dataset
# NOTE: Private data can be accessed by the Data Owners, but NOT the Data Scientists
real = domain_client.datasets[0].assets[0].data

In [ ]:
assert ca_data.equals(real)

### Create account for Data Scientist user

Signup is disabled by default.
An Admin/DO can enable it by `domain_client.settings.allow_guest_signup(enable=True)`

Refer to notebook [07-domain-register-control-flow](./07-domain-register-control-flow.ipynb) for more information.

In [ ]:
domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")

In [ ]:
# Cleanup local domain

if node.node_type.value == "python":
    node.land()